## CIFAR10 Hparam search

In [2]:
import torch
import numpy as np
import os, sys
import shutil
import pickle
import cv2
from tqdm import tqdm
from sklearn.cluster import KMeans
from datetime import datetime

sys.path.insert(0, 'src')
from utils.utils import ensure_dir, read_json, informal_log
from utils.visualizations import plot
from utils.model_utils import prepare_device

import model.metric as module_metric
import model.loss as module_loss
import datasets.datasets as module_data
import model.model as module_model

from src.train import main as train_fn
from predict import predict
from parse_config import ConfigParser


sys.path.insert(0, 'setup')
from setup_cifar10 import setup_cifar10 
# import cv2
# print(cv2.__version__)

In [3]:
config_path = 'configs/train_cifar10_pixel_explainer.json'
debug = False
if debug:
    learning_rates = [1e-6] #, 1e-5, 1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1] #, 1e-2, 1e-3]
else:
    learning_rates = [1e-4, 1e-3, 5e-2, 1e-2, 5e-1, 1e-1]
    weight_decays = [0, 1e-1, 1e-2, 1e-3]

config_json = read_json(config_path)


### Set up data loaders

In [4]:
dataset_args = config_json['dataset']['args']
train_descriptors_dataset = module_data.KDDataset(split='train', **dataset_args)
test_descriptors_dataset = module_data.KDDataset(split='test', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_descriptors_dataloader = torch.utils.data.DataLoader(
    train_descriptors_dataset,
    shuffle=True,
    **dataloader_args)
test_descriptors_dataloader = torch.utils.data.DataLoader(
    test_descriptors_dataset,
    shuffle=False,
    **dataloader_args)

device, device_ids = prepare_device(config_json['n_gpu'])
metric_fns = [getattr(module_metric, met) for met in config_json['metrics']]
loss_fn = getattr(module_loss, config_json['loss'])

### Run Hyperparameter search

In [5]:
best = {
    'lr': -1,
    'wd': -1,
    'val_acc': -1
}
n_trials = len(learning_rates) * len(weight_decays)
trial_idx = 1
timestamp = datetime.now().strftime(r'%m%d_%H%M%S')

# Logging
log_path = os.path.join(config_json['trainer']['save_dir'], timestamp, 'log.txt')
ensure_dir(os.path.dirname(log_path))
informal_log("Hyperparameter search", log_path)
informal_log("Learning rates: {}".format(learning_rates), log_path)
informal_log("Weight decays: {}".format(weight_decays), log_path)

# 

# Debug mode
if debug:
    config_json['trainer']['epochs'] = 1
    
for lr in learning_rates:
    for wd in weight_decays:
        # Update config json
        config_json['optimizer']['args'].update({
            'lr': lr,
            'weight_decay': wd
        })
        
        # Create run ID for trial
        itr_timestamp = datetime.now().strftime(r'%m%d_%H%M%S')
        informal_log("[{}] Trial {}/{}: LR = {} WD = {}".format(
            itr_timestamp, trial_idx, n_trials, lr, wd), log_path)
        run_id = os.path.join(timestamp, 'trials', 'lr_{}-wd_{}'.format(lr, wd))
        config = ConfigParser(config_json, run_id=run_id)
        print(config.config['optimizer']['args'])
        
        # Train model
        model = train_fn(
            config=config, 
            train_data_loader=train_descriptors_dataloader,
            val_data_loader=test_descriptors_dataloader)
        
        # Restore model
        model_restore_path = os.path.join(config.save_dir, 'model_best.pth')
        
        model.restore_model(model_restore_path)
        print("restored model")
        # Run on validation set using predict function
        
        trial_path = os.path.dirname(os.path.dirname(model_restore_path))
        output_save_path = os.path.join(trial_path, "val_outputs.pth")
        log_save_path = os.path.join(trial_path, "val_metrics.pth")
        
        validation_data = predict(
            data_loader=test_descriptors_dataloader,
            model=model,
            metric_fns=metric_fns,
            device=device,
            loss_fn=loss_fn,
            output_save_path=output_save_path,
            log_save_path=log_save_path)
       
        # Obtain accuracy and compare to previous best
        print(validation_data['metrics'].keys())
        val_accuracy = validation_data['metrics']['accuracy']
        if val_accuracy > best['val_acc']:
            best.update({
                'lr': lr,
                'wd': wd,
                'val_acc': val_accuracy
            })
            informal_log("Best accuracy of {:.3f} with lr={} and wd={}".format(val_accuracy, lr, wd), log_path)
            informal_log("Trial path: {}".format(trial_path), log_path)
            # Copy model and outputs to 1 directory for easy access
            best_save_dir = os.path.join(os.path.dirname(os.path.dirname(trial_path)), 'best')
            ensure_dir(best_save_dir)
            best_outputs_save_path = os.path.join(best_save_dir, 'outputs.pth')
            best_model_save_path = os.path.join(best_save_dir, 'model.pth')
            torch.save(validation_data['logits'], best_outputs_save_path)
            model.save_model(best_model_save_path)
            informal_log("Saved model and outputs to {}".format(best_save_dir), log_path)
            
            
        trial_idx += 1

Hyperparameter search
Learning rates: [0.0001, 0.001, 0.05, 0.01, 0.5, 0.1]
Weight decays: [0, 0.1, 0.01, 0.001]
[0627_150603] Trial 1/24: LR = 0.0001 WD = 0
OrderedDict([('lr', 0.0001), ('weight_decay', 0)])
Created LinearLayers model with 30730 trainable parameters
Training from scratch.
Checkpoint save directory: saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.0001-wd_0/models
Train Epoch: 1 [0/196 (0%)] Loss: 2.380296
Train Epoch: 1 [48/196 (24%)] Loss: 2.304628
Train Epoch: 1 [96/196 (49%)] Loss: 2.279899
Train Epoch: 1 [144/196 (73%)] Loss: 2.148456
Train Epoch: 1 [192/196 (98%)] Loss: 2.210256
    epoch          : 1
    val_TP         : [350 150  42  75 257 130 287 112 267 392]
    val_TN         : [7974 8138 8675 8643 8115 8506 7876 8637 8046 7452]
    val_FPs        : [1010  872  323  365  878  492 1143  377  936 1542]
    val_FNs        : [666 840 960 917 750 872 694 874 751 614]
    val_accuracy   : 0.2062
    val

100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 90.72it/s]


Saving validation results to saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.0001-wd_0
restored model


100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 96.58it/s]

dict_keys(['TP', 'TN', 'FPs', 'FNs', 'accuracy', 'RMSE', 'per_class_accuracy', 'per_class_accuracy_mean', 'precision', 'precision_mean', 'recall', 'recall_mean', 'predicted_class_distribution', 'f1', 'f1_mean'])
Best accuracy of 0.392 with lr=0.0001 and wd=0
Trial path: saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.0001-wd_0
Saved model and outputs to saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/best
[0627_150853] Trial 2/24: LR = 0.0001 WD = 0.1
OrderedDict([('lr', 0.0001), ('weight_decay', 0.1)])
Created LinearLayers model with 30730 trainable parameters
Training from scratch.
Checkpoint save directory: saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.0001-wd_0.1/models


...

Train Epoch: 1 [0/196 (0%)] Loss: 2.380296
Train Epoch: 1 [48/196 (24%)] Loss: 5.929747
Train Epoch: 1 [96/196 (49%)] Loss: 8.243635
Train Epoch: 1 [144/196 (73%)] Loss: 6.633879
Train Epoch: 1 [192/196 (98%)] Loss: 8.320148
    epoch          : 1
    val_TP         : [384 182 104  51 611  80 105 463 273  47]
    val_TN         : [8389 8729 8748 8862 5544 8793 8820 6744 8721 8950]
    val_FPs        : [ 595  281  250  146 3449  205  199 2270  261   44]
    val_FNs        : [632 808 898 941 396 922 876 523 745 959]
    val_accuracy   : 0.23
    val_RMSE       : 3.411290078547997
    val_per_class_accuracy: [0.8773 0.8911 0.8852 0.8913 0.6155 0.8873 0.8925 0.7207 0.8994 0.8997]
    val_per_class_accuracy_mean: 0.8459999999999999
    val_precision  : [0.39223698 0.39308855 0.29378531 0.25888325 0.15049261 0.28070175
 0.34539474 0.1694109  0.51123596 0.51648352]
    val_precision_mean: 0.33117135662617914
    val_recall     : [0.37795276 0.18383838 0.10379242 0.05141129 0.60675273 0.079840

100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 88.05it/s]


Saving validation results to saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.1-wd_0.001
restored model


100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 91.76it/s]

dict_keys(['TP', 'TN', 'FPs', 'FNs', 'accuracy', 'RMSE', 'per_class_accuracy', 'per_class_accuracy_mean', 'precision', 'precision_mean', 'recall', 'recall_mean', 'predicted_class_distribution', 'f1', 'f1_mean'])


### Check best model's results on training data

In [7]:
# Trained on soft labels with 75 clusters
# model_restore_path = 'saved/cifar10/resnet18/explainer/sift_descriptor_histogram/hparam_search/0525_103130/best/model.pth'
# Trained on hard GT labels with 75 clusters
# model_restore_path = 'saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.001-wd_0.1/models/model_best.pth'
# Variable
restore_dir = 'saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.001-wd_0.1'
model_restore_path = os.path.join(restore_dir, 'models/model_best.pth')
config_json = read_json(os.path.join(restore_dir, 'models/config.json'))

device, device_ids = prepare_device(config_json['n_gpu'])
metric_fns = [getattr(module_metric, met) for met in config_json['metrics']]
loss_fn = getattr(module_loss, config_json['loss'])

model_args = config_json['arch']['args']
model = module_model.LinearLayers(
    checkpoint_path=model_restore_path,
    **model_args)

print(model)

model.eval()
model = model.to(device)

# Dataloaders
dataset_args = config_json['dataset']['args']
train_descriptors_dataset = module_data.KDDataset(split='train', **dataset_args)
test_descriptors_dataset = module_data.KDDataset(split='test', **dataset_args)

dataloader_args = config_json['data_loader']['args']
train_descriptors_dataloader = torch.utils.data.DataLoader(
    train_descriptors_dataset,
    shuffle=True,
    **dataloader_args)
test_descriptors_dataloader = torch.utils.data.DataLoader(
    test_descriptors_dataset,
    shuffle=False,
    **dataloader_args)



# Evaluate on validation set as a sanity check
validation_results = predict(
        data_loader=test_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Validation accuracy: {}".format(validation_results['metrics']['accuracy']))

training_results = predict(
        data_loader=train_descriptors_dataloader,
        model=model,
        metric_fns=metric_fns,
        device=device,
        loss_fn=loss_fn,
        output_save_path=None,
        log_save_path=None)

print("Training accuracy: {}".format(training_results['metrics']['accuracy']))

LinearLayers(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=10, bias=True)
  )
)
Trainable parameters: 30730


100%|███████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 92.79it/s]


Validation accuracy: 0.4136


100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 236.18it/s]

Training accuracy: 0.42728


In [8]:
# Save outputs of explainer on train and val
save_path = os.path.join(restore_dir, 'outputs_predictions.pth')

val_outputs = validation_results['logits']
val_probabilities = torch.softmax(val_outputs, dim=1)
val_predictions = torch.argmax(val_outputs, dim=1)
val_outputs_predictions = {
    'outputs': val_outputs.cpu().numpy(),
    'probabilities': val_probabilities.cpu().numpy(),
    'predictions': val_predictions.cpu().numpy()
}

train_outputs = training_results['logits']
train_probabilities = torch.softmax(train_outputs, dim=1)
train_predictions = torch.argmax(train_outputs, dim=1)

train_outputs_predictions = {
    'outputs': train_outputs.cpu().numpy(),
    'probabilities': train_probabilities.cpu().numpy(),
    'predictions': train_predictions.cpu().numpy()
}

outputs_predictions = {
    'train': train_outputs_predictions,
    'test': val_outputs_predictions
}
if not os.path.exists(save_path):
    torch.save(outputs_predictions, save_path)
    print("Saved outputs, probabilities and predictions for train/val to {}".format(save_path))
else:
    print("File exists at {}".format(save_path))

Saved outputs, probabilities and predictions for train/val to saved/cifar10/resnet18/explainer/pixels/model_soft_labels/hparam_search/cross_entropy/0627_150603/trials/lr_0.001-wd_0.1/outputs_predictions.pth
